In [1]:
!pip install pandas xlrd ipdb

In [2]:
import os
import pandas as pd
import ipdb
from datetime import datetime, time

In [3]:
%ls data

weekly-2020-02-10 0925.xlsx


In [4]:
libs = sorted(pd.read_csv('libraries.csv').full_name.tolist())

In [5]:
# CONVERT
dn = 'data'
fn = dn + '/' + 'weekly-2020-02-10 0925.xlsx'
xl = pd.ExcelFile(fn)
sheets = xl.sheet_names
res=[]
for sheet in sheets:
    df = xl.parse(sheet)
    date = datetime.strptime(df.columns[0], '%d %B %Y')
    df = (
        df.loc[df['Counter Name'].isin(libs)]
        .rename(columns={df.columns[0]: 'dt'})
        .fillna('x')        
    )
    times=df.loc[df.dt!='x']
    df.dt = df.dt.replace(to_replace='x', method='ffill')
    dt = list(map(lambda x: datetime.combine(date, time(int(x[:2]), 0)), df.dt.tolist()))
    df.dt = dt
    res.append(df)                             

df = pd.concat(res)
df = df.rename(columns={'Counter Name': 'Library'})

In [6]:
def to_hours(x):
    return pd.to_datetime(x).hour

oh = pd.read_csv('opening hours.csv', converters={'start': lambda x: to_hours(x), 'finish': lambda x: to_hours(x)})
oh

,library,day,start,finish,opening_type
0,Burnt Oak,Mon,14,20,staffed
1,Burnt Oak,Wed,9,12,staffed
2,Burnt Oak,Fri,14,17,staffed
3,Burnt Oak,Sat,14,17,staffed
4,Burnt Oak,Mon,9,14,SSO
...,...,...,...,...,...
144,North Finchley,Thu,12,20,SSO
145,North Finchley,Fri,17,20,SSO
146,East Finchley,Thu,17,20,SSO
147,Osidge,Thu,17,20,SSO


In [26]:
check = df.iloc[246][['dt', 'Library']]
def get_ot(dt, lib):
    import ipdb
    in_df = pd.concat([dt, lib], axis=1)
    res=[]
    for row in in_df.itertuples():
#         ipdb.set_trace()
        day = row.dt.day_name()[:3]
        mask = (
            (oh.library==row.Library) &
            (oh.day==day) &
            (oh.start<=row.dt.hour) &
            (oh.finish>row.dt.hour)
        )
        if oh.loc[mask].empty:
            res.append('closed')
        else:
            ot = oh.loc[mask].opening_type.values[0]
            res.append(ot)
#     ipdb.set_trace()
    return pd.Series(res)

ots = get_ot(df.dt, df.Library)
df['opening_type'] = ots.values
df

,dt,Library,People In,People Out,opening_type
0,2020-02-03 00:00:00,Chipping Barnet,0,0,closed
6,2020-02-03 00:00:00,Colindale,0,0,closed
9,2020-02-03 00:00:00,East Finchley,0,0,closed
12,2020-02-03 00:00:00,Edgware,0,0,closed
15,2020-02-03 00:00:00,Golders Green,0,0,closed
...,...,...,...,...,...
561,2020-02-09 23:00:00,East Finchley,0,0,closed
564,2020-02-09 23:00:00,Edgware,0,0,closed
567,2020-02-09 23:00:00,Golders Green,0,0,closed
570,2020-02-09 23:00:00,North Finchley,0,0,closed


In [20]:
%debug

> /Users/richardlynch/opt/anaconda3/lib/python3.7/site-packages/pandas/core/ops/__init__.py(1142)wrapper()
   1140 
   1141         elif isinstance(other, ABCSeries) and not self._indexed_same(other):
-> 1142             raise ValueError("Can only compare identically-labeled " "Series objects")
   1143 
   1144         elif is_categorical_dtype(self):

ipdb> oh.start
*** NameError: name 'oh' is not defined
ipdb> start
*** NameError: name 'start' is not defined
ipdb> self.start
*** AttributeError: 'Series' object has no attribute 'start'
ipdb> self
0           Burnt Oak
1           Burnt Oak
2           Burnt Oak
3           Burnt Oak
4           Burnt Oak
            ...      
144    North Finchley
145    North Finchley
146     East Finchley
147            Osidge
148            Osidge
Name: library, Length: 149, dtype: object
ipdb> quit
